In [30]:
import requests
response = requests.get('https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating')

In [29]:
from bs4 import BeautifulSoup
doc = BeautifulSoup(response.text, 'html.parser')

In [76]:
def getTitles(doc):
    movie_titles = []
    class_name = 'lister-item-header' #from the source code 
    all_title_tags = doc.find_all('h3', {'class': class_name})
    for tags in all_title_tags:
        title = tags.find('a').text
        movie_titles.append(title)
    return movie_titles

In [64]:
def getYears(doc):
    movie_year = []
    class_name = 'lister-item-year text-muted unbold' #from the source code 
    all_year_tags = doc.find_all('span', {'class': class_name})
    for tags in all_year_tags:
        year = tags.text.strip()[1:5]
        movie_year.append(year)
    return movie_year

In [72]:
def getGenres(doc):
    movie_genre = []
    class_name = 'genre' #from the source code 
    all_genre_tags = doc.find_all('span', {'class': class_name})
    for tags in all_genre_tags:
        genre = tags.text.strip()
        movie_genre.append(genre)
    return movie_genre
print(getGenres(doc))

['Drama', 'Crime, Drama', 'Action, Crime, Drama', 'Action, Adventure, Drama', 'Biography, Drama, History', 'Crime, Drama', 'Crime, Drama', 'Adventure, Comedy, Drama', 'Crime, Drama', 'Biography, Drama', 'Crime, Drama, Mystery', 'Action, Adventure, Sci-Fi', 'Action, Adventure, Drama', 'Drama', 'Action, Adventure, Drama', 'Drama, Romance', 'Adventure, Western', 'Drama', 'Action, Sci-Fi', 'Biography, Crime, Drama', 'Action, Adventure, Fantasy', 'Drama', 'Action, Drama, Mystery', 'Adventure, Drama, Sci-Fi', 'Crime, Drama', 'Animation, Adventure, Family', 'Drama, War', 'Crime, Drama, Fantasy', 'Comedy, Drama, Romance', 'Crime, Drama, Mystery', 'Action, Sci-Fi', 'Crime, Drama, Thriller', 'Action, Adventure, Fantasy', 'Action, Drama, Mystery', 'Action, Drama', 'Drama, Family, Fantasy', 'Action, Adventure, Crime', 'Drama, Romance', 'Drama, Thriller', 'Drama, Music', 'Biography, Comedy, Drama', 'Action, Adventure, Drama', 'Drama, Mystery, Sci-Fi', 'Crime, Drama, Thriller', 'Biography, Drama, Mu

In [85]:
def getCertifications(doc):
    movie_certification = []
    movie_tags = doc.find_all('div',{'class':'lister-item-content'})
    for tags in movie_tags:
        certification_tags = tags.find('span', {'class': 'certificate'})
        if certification_tags:
            movie_certification.append(certification_tags.text)
        else:
            movie_certification.append('NA')
    return movie_certification
print(getCertifications(doc))


['R', 'R', 'PG-13', 'PG-13', 'R', 'R', 'Approved', 'NA', 'R', 'NA', 'TV-MA', 'PG-13', 'PG-13', 'R', 'PG-13', 'PG-13', 'R', 'TV-MA', 'R', 'R', 'PG', 'R', 'NA', 'PG-13', 'R', 'PG', 'R', 'R', 'PG-13', 'R', 'R', 'R', 'PG', 'Not Rated', 'Not Rated', 'PG', 'Not Rated', 'Not Rated', 'R', 'R', 'R', 'NA', 'PG-13', 'R', 'R', 'R', 'R', 'R', 'R', 'G', 'R', 'Not Rated', 'PG', 'R', 'R', 'PG-13', 'R', 'PG', 'PG', 'G', 'G', 'Not Rated', 'NA', 'PG-13', 'R', 'R', 'PG', 'PG', 'PG-13', 'PG-13', 'PG', 'R', 'PG-13', 'PG-13', 'NA', 'PG-13', 'TV-14', 'TV-MA', 'PG', 'G', 'R', 'R', 'R', 'R', 'PG-13', 'R', 'Not Rated', 'R', 'R', 'PG', 'R', 'R', 'Not Rated', 'PG', 'Approved', 'Approved', 'Passed', 'G', 'Not Rated', 'Not Rated']


In [63]:
def getDurations(doc):
    movie_duration = []
    class_name = 'runtime' #from the source code 
    all_duration_tags = doc.find_all('span', {'class': class_name})
    for tags in all_duration_tags:
        duration = tags.text.strip()[0:3]
        movie_duration.append(duration)
    return movie_duration


In [67]:
def getRatings(doc):
    movie_rating = []
    class_name = 'inline-block ratings-imdb-rating' #from the source code 
    all_rating_tags = doc.find_all('div', {'class': class_name})
    for tags in all_rating_tags:
        rating = tags.find('strong').text
        movie_rating.append(rating)
    return movie_rating

In [70]:
def getUrls(doc):
    movie_url = []
    class_name = 'lister-item-header' #from the source code 
    all_url_tags = doc.find_all('h3', {'class': class_name})
    for tags in all_url_tags:
        url = 'https://www.imdb.com/' + tags.find('a')['href']
        movie_url.append(url)
    return movie_url

In [88]:
import pandas as pd
def allMovies():
    movies_dict={
        'title':[],
        'genre':[],
        'duration':[],
        'rating':[],
        'year':[],
        'certification':[],
        'url':[]
    }
    for i in range(1,2000,100):
        try:
            url = 'https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start='+str(i)+'&ref_=adv_next'
            response = requests.get(url)
        except:
            break
        
        if response.status_code != 200:
            break
           
        doc = BeautifulSoup(response.text, 'html.parser')
        titles = getTitles(doc)
        urls = getUrls(doc)
        certifications = getCertifications(doc)
        ratings = getRatings(doc)
        durations = getDurations(doc)
        years = getYears(doc)
        genres = getGenres(doc)
    
        
        for i in range(len(titles)):
            movies_dict['title'].append(titles[i])
            movies_dict['genre'].append(genres[i])
            movies_dict['duration'].append(durations[i])
            movies_dict['rating'].append(ratings[i])
            movies_dict['year'].append(years[i])
            movies_dict['certification'].append(certifications[i])
            movies_dict['url'].append(urls[i])
        
    return pd.DataFrame(movies_dict)

In [89]:
movies = allMovies()
movies.to_csv('movies.csv', index = None)